In [20]:
import pandas as pd

In [21]:
with open("mtg/data/autotagging_genre.tsv") as src:
    with open("dataset/mtg.tsv", "w") as targ:
        for line in src.readlines():
            fields = line.split("\t")
            fields = fields[:5] + [','.join(fields[5:])]
            targ.write('\t'.join(fields) + "\n")

In [22]:
df = pd.read_csv("dataset/mtg.tsv", sep="\t")

In [23]:
df['TAGS'] = df['TAGS'].apply(lambda x: [y.removeprefix("genre---") for y in x.split(",")])

In [24]:
df = df[df['TAGS'].apply(lambda x: len(x) > 0)]

In [25]:
df.loc[:, 'GENRE'] = df['TAGS'].apply(lambda x: x[0])

In [26]:
df.loc[:, 'BATCH'] = df['PATH'].apply(lambda x: int(x.split('/')[0]))

In [27]:
df = df[df['BATCH'] <= 18] # I ran out of SSD after 18 tarballs, but I think it's enough.

In [28]:
drop_classes = ['easylistening', 'chillout']
df = df[~df['GENRE'].isin(drop_classes)]

In [29]:
good_genres = set(df['GENRE'].value_counts()[:10].index)

In [30]:
df = df[df['TAGS'].apply(lambda x: len(set(x) & good_genres) > 0)]

In [31]:
df.to_csv("mtg/selected-tracks.csv", index=None)

In [33]:
!mkdir dataset/mtg/extracted

mkdir: dataset/mtg/extracted: File exists


In [34]:
import tarfile

for paths in df.groupby('BATCH')['PATH'].agg(list):
    batch = paths[0].split("/")[0]
    # Open the tar file
    with tarfile.open(f"dataset/mtg/raw_30s_audio-{batch}.tar", "r") as tar:
        for p in paths:
            # Extract a single file
            tar.extract(p, path=f"dataset/mtg/extracted")

/var/folders/c7/l787n6sd6db5qx890vglg8_m0000gn/T/ipykernel_96145/570987002.py:9: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extract(p, path=f"dataset/mtg/extracted")
